In [1]:
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
import pandas as pd

import numpy as np 

import create_model
import pickle
import time 
import copy
import random
import data_formatting


def validate(train):
    
    if train == True:
        
        model = train_model
        mode = 'TRAIN'
        
    else:
        mode = 'DEV'
        model = dev_model

    encoder, decoder, predicted = session.run([model.encoder_inputs, model.decoder_targets, model.decoder_pred_train])            

    print ('Current mode:%s' % mode)
    for i, (e_in, dt_targ, dt_pred) in enumerate(zip( encoder, decoder, predicted)):

        print('  sample {}:'.format(i + 1))
        #print('    enc input           > {}'.format(e_in))
        print('    enc input           > {}'.format(data_formatting.decodeSent(e_in, inv_map)))

        #print('    dec input           > {}'.format(dt_targ))
        print('    dec input           > {}'.format(data_formatting.decodeSent(dt_targ, inv_map)))

        #print('    dec train predicted > {}'.format(dt_pred))
        print('    dec train predicted > {}'.format(data_formatting.decodeSent(dt_pred, inv_map)))

        if i >= 0: break

In [2]:
#dataset = 'enron'

#vocab_dict = pickle.load(open('../processed_data/word_dict_v02_enron_py35_seq_length_3_49_sample_4256_limited_vocab.pkl', 'rb'))
#df_all = pd.read_pickle('../processed_data/processed_data_v02_enron_py35_seq_length_3_49_sample_4256_limited_vocab.pkl')

dataset = 'twitter'

#vocab_dict = pickle.load(open('../processed_data/word_dict_v02_twitter_py35_seq_length_3_25_sample_1764604_questions.pkl', 'rb'))
#df_all = pd.read_pickle('../processed_data/processed_data_v02_twitter_py35_seq_length_3_25_sample_1764604_questions.pkl')

#vocab_dict = pickle.load(open('../processed_data/word_dict_v02_twitter_py35_seq_length_4_15_sample_134241_full.pkl', 'rb'))
#df_all = pd.read_pickle('../processed_data/processed_data_v02_twitter_py35_seq_length_4_15_sample_134241_full.pkl')

vocab_dict = pickle.load(open('../processed_data/word_dict_v02_twitter_py35_seq_length_3_19_sample_21946_lem.pkl', 'rb'))
df_all = pd.read_pickle('../processed_data/processed_data_v02_twitter_py35_seq_length_3_19_sample_21946_lem.pkl')

#vocab_dict = pickle.load(open('../processed_data/word_dict_v02_twitter_py35_seq_length_3_25_sample_1901567_full.pkl', 'rb'))
#df_all = pd.read_pickle('../processed_data/processed_data_v02_twitter_py35_seq_length_3_25_sample_1901567_full.pkl')

In [3]:
inv_map = data_formatting.createInvMap(vocab_dict)

In [30]:
train_model_params = {'n_cells':256, 'num_layers':2, 'embedding_size':512, 
          'vocab_size':len(vocab_dict) + 1, 'minibatch_size':64, 'n_threads':128,
          'beam_width':10, 'limit_decode_steps': None,
          'encoder_input_keep':1, 'decoder_input_keep':1,
          'encoder_output_keep':1, 'decoder_output_keep':1, 'anti_lm_weight':0.5,
                      'anti_lm_max_step':3
         }

In [31]:
dev_model_params = train_model_params
dev_model_params['encoder_input_keep'] = 1
dev_model_params['encoder_output_keep'] = 1
dev_model_params['decoder_input_keep'] = 1
dev_model_params['decoder_output_keep'] = 1

training_params = { 'vocab_lower':3, 'vocab_upper':train_model_params['vocab_size']-1, 
                    'n_epochs':500000}

In [32]:
df_all['alpha_Pair_0_encoding'] = df_all['alpha_Pair_0_tokens'].apply(lambda x: data_formatting.encodeSent(x, vocab_dict))
df_all['alpha_Pair_1_encoding'] = df_all['alpha_Pair_1_tokens'].apply(lambda x: data_formatting.encodeSent(x, vocab_dict))
df_all['Index'] = df_all.index.values

df_all_train = df_all.sample(frac=0.97, random_state=1)

df_all_dev = df_all[df_all['Index'].isin(df_all_train['Index'].values) == False]

df_all_test = df_all_dev.sample(frac=0.10, random_state=1)

df_all_dev = df_all_dev[df_all_dev['Index'].isin(df_all_test['Index'].values) == False]

print (df_all.shape[0], df_all_train.shape[0],  df_all_dev.shape[0], df_all_test.shape[0], len(vocab_dict))

train_data = data_formatting.prepare_train_batch(df_all_train['alpha_Pair_0_encoding'].values, 
                                                    df_all_train['alpha_Pair_1_encoding'].values)

dev_data = data_formatting.prepare_train_batch(df_all_dev['alpha_Pair_0_encoding'].values, 
                                                    df_all_dev['alpha_Pair_1_encoding'].values)

test_data = data_formatting.prepare_train_batch(df_all_test['alpha_Pair_0_encoding'].values, 
                                                    df_all_test['alpha_Pair_1_encoding'].values)

17690 17159 478 53 17322


In [33]:
tf.reset_default_graph()
with tf.variable_scope('training_model'):
    
    train_model = create_model.Model(train_model_params, 'train', train_data)

with tf.variable_scope('training_model', reuse=True):

    dev_model = create_model.Model(dev_model_params, 'train', dev_data)   

train
train


In [34]:
global_step = tf.Variable(0, trainable=False)

starter_learning_rate = tf.placeholder(tf.float32, shape=(), name='starter_learning_rate')

learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, \
                                   training_params['n_epochs'], 0.98, staircase=False)

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

train_op = optimizer.minimize(train_model.loss, global_step=global_step)

In [ ]:
print_interval = 100
save_interval = 1000

lr = 0.001

train_loss = []
dev_loss = []

train_accuracy = []
dev_accuracy = []

learning_rate = []

with tf.Session() as session:

    session.run(tf.global_variables_initializer())
    saver = tf.train.Saver()

    coord = tf.train.Coordinator()

    threads = tf.train.start_queue_runners(coord=coord)
    #saver.restore(session, 'chkpt/seq2seq_twitter_test_2-5001')


    for epoch in range(training_params['n_epochs']):

        start_time = time.time()

        session.run(train_op, feed_dict={'starter_learning_rate:0':lr})
        #session.run(train_op)

        if epoch % print_interval == 0: 

            print ('epoch:%d, global_step:%s, learning rate:%.3g' % 
                       (epoch, tf.train.global_step(session, global_step), 
                        session.run(optimizer._lr, feed_dict={'starter_learning_rate:0':lr})))

            train_minibatch_loss,  train_minibatch_accuracy = session.run([train_model.loss, train_model.accuracy], 
                                                                          feed_dict={'starter_learning_rate:0':lr})

            train_loss.append([tf.train.global_step(session, global_step), train_minibatch_loss])  
            train_accuracy.append([tf.train.global_step(session, global_step), train_minibatch_accuracy])  

            print ('training minibatch loss:%.6g' % (train_minibatch_loss))
            print ('training minibatch accuracy:%.6g' % (train_minibatch_accuracy))

            validate(train=True)

            dev_model_loss, dev_model_accuracy = session.run([dev_model.loss, dev_model.accuracy])            

            print ('dev minibatch loss:%.6g' %  (dev_model_loss))
            print ('dev minibatch accuracy:%.6g' %  (dev_model_accuracy))

            dev_loss.append([tf.train.global_step(session, global_step), dev_model_loss])

            dev_accuracy.append([tf.train.global_step(session, global_step), dev_model_accuracy])

            learning_rate.append([tf.train.global_step(session, global_step), 
                                  session.run(optimizer._lr, feed_dict={'starter_learning_rate:0':lr})])

            validate(train=False)

            print ('Epoch:%d finished, time:%.4g' % (epoch, time.time() - start_time))

        if (epoch % save_interval == 0):# & (epoch!=0): 

            saver.save(session, '../chkpt/seq2seq_twitter_antilm_precise', global_step = tf.train.global_step(session, global_step))

            print ('Session saved')

    coord.request_stop()
    coord.join(threads)

epoch:0, global_step:1, learning rate:0.001
training minibatch loss:9.72985
training minibatch accuracy:0.0120773
Current mode:TRAIN
  sample 1:
    enc input           > ['so', '?', '?', '?', 'hillary', 's', 'mentor', 'senator', 'byrd', 'wa', 'a', 'dragon', 'for', 'kkk', '<EOS>']
    dec input           > ['is', 'the', 'picture', 'fake', '?', 'you', 'no', 'is', 'not', 'is', 'a', 'true', 'picture', 'and', 'a', 'fact', '<EOS>']
    dec train predicted > ['backpage', 'backpage', 'backpage', 'backpage', 'backpage', 'backpage', '<EOS>', '<EOS>', '<EOS>', '<EOS>', 'backpage', 'trek', '<EOS>', 'suicide', 'swears', 'swears', 'er']
dev minibatch loss:9.72967
dev minibatch accuracy:0.0136799
Current mode:DEV
  sample 1:
    enc input           > ['oh', 'please', 'i', 'm', 'trying', 'to', 'keep', 'up', 'with', 'you', '<EOS>']
    dec input           > ['so', 'proud', 'of', 'my', 'friend', 'and', 'amazing', 'journalist', 'for', 'winning', 'an', 'emmy', 'who', 'are', 'you', '?', 'haha', '<EOS>']
 

dev minibatch loss:5.69666
dev minibatch accuracy:0.22486
Current mode:DEV
  sample 1:
    enc input           > ['yes', 'they', 'can', 'it', 's', 'happened', 'before', '<EOS>']
    dec input           > ['don', 't', 'think', 'both', 'sec', 'on', 'cbs', 'and', 'college', 'gameday', 'can', 'be', 'there', '<EOS>']
    dec train predicted > ['i', 't', 'be', 'that', '?', 'endorse', 'the', 'of', 'victim', '<EOS>', '<EOS>', 'you', 'a', '<EOS>']
Epoch:900 finished, time:1.44
epoch:1000, global_step:1001, learning rate:0.001
training minibatch loss:4.20113
training minibatch accuracy:0.276301
Current mode:TRAIN
  sample 1:
    enc input           > ['horrible', 'what', 'happened', 'to', 'bochy', '?', 'why', 'would', 'you', 'take', 'madbum', 'out', 'with', 'a', 'hitter', 'and', 'pitch', '<EOS>']
    dec input           > ['i', 'literally', 'could', 'not', 'bring', 'myself', 'to', 'watch', 'and', 'thankfully', 'so', '<EOS>']
    dec train predicted > ['i', 'm', 'found', 'have', 'a', 'to', 'for',

In [9]:
from tensorflow.python.framework import ops
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import control_flow_ops
from tensorflow.python.framework import dtypes
from tensorflow.python.ops import nn_ops
from tensorflow.python.framework import tensor_util
from tensorflow.python.framework import tensor_shape
from tensorflow.python.util import nest
from tensorflow.python.ops import math_ops
from tensorflow.python.ops import tensor_array_ops

In [20]:
tf.reset_default_graph()

inf_model_1_params = {'n_cells':256, 'num_layers':2, 'embedding_size':512, 
          'vocab_size':len(vocab_dict) + 1, 'minibatch_size':64, 'n_threads':128,
          'beam_width':1, 'limit_decode_steps': None,
          'encoder_input_keep':1, 'decoder_input_keep':1,
          'encoder_output_keep':1, 'decoder_output_keep':1,
         }

inf_model_2_params = {'n_cells':256, 'num_layers':2, 'embedding_size':512, 
          'vocab_size':len(vocab_dict) + 1,
          'minibatch_size':64, 'n_threads':128,
          'beam_width':2, 'limit_decode_steps': None,
          'encoder_input_keep':1, 'decoder_input_keep':1,
          'encoder_output_keep':1, 'decoder_output_keep':1, 'anti_lm_weight':0,
           'anti_lm_max_step':3
         }

#with tf.variable_scope('training_model'):

#    inf_model_1 = create_model.Model(inf_model_1_params, 'infer', None)
    
with tf.variable_scope('training_model'):

    inf_model_2 = create_model.Model(inf_model_2_params, 'infer', None)   

#with tf.variable_scope('training_model'):

    #inf_train = create_model.Model(inf_model_2_params, 'debug', None)   

In [21]:
df_all_train.values[0]

array([list(['yeah', 'i', 'am', 'what', 'the', 'fuc', '<EOS>']),
       list(['good', 'taste', 'also', 'ur', 'location', 'is', 'on', 'are', 'u', 'in', 'ny', 'rn', '?', '?', '<EOS>']),
       list([15653, 6178, 7228, 10036, 7441, 8452, 1]),
       list([5414, 3593, 16021, 14506, 14758, 9503, 7255, 5123, 14290, 14250, 4970, 15839, 10150, 10150, 1]),
       208888], dtype=object)

In [22]:
input_user = data_formatting.encodeSent(df_all_train.values[0][0], vocab_dict)
#input_user = data_formatting.encodeSent(['i', 'll', 'be', 'there', 'tomorrow', '<EOS>'], vocab_dict)

with tf.Session() as sess:
    #sess.run(tf.global_variables_initializer())
    #tf.set_random_seed(1)
    saver = tf.train.Saver()
   
    saver.restore(sess, '../chkpt/seq2seq_twitter_testing-5501')
    q_0 = sess.run(
        #inf_train.final_outputs, 
        inf_model_2.decoder_pred_decode,
        feed_dict={'training_model/encoder_inputs:0':[input_user], 
                         'training_model/encoder_inputs_length:0':[len(input_user)]})

INFO:tensorflow:Restoring parameters from ../chkpt/seq2seq_twitter_testing-5501


In [28]:
data_formatting.decodeSent(list(zip(*q_0[0]))[0], inv_map)

['lmfao', 'matt', 'can', 't', 'figure', 'me', 'this', '<EOS>']

In [13]:
np.array([[data_formatting.decodeSent(q_0.predicted_ids.T[i][0], inv_map), 
          q_0.scores.T[i][0]]for i in range(inf_model_2_params['beam_width'])])

array([[ list(['lmfao', 'matt', 'can', 't', 'figure', 'me', 'being', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>', '<EOS>']),
        array([-0.30946836, -0.59456718, -1.12294447, -1.54447973, -1.60471201,
       -1.72047746, -1.94635522, -3.12297106, -3.81752348, -3.81752348,
       -3.81752348, -3.81752348, -3.81752348, -3.81752348, -3.81752348,
       -3.81752348, -3.81752348, -3.81752348], dtype=float32)],
       [ list(['lmao', 'lt', 'could', 'be', 'back', 'out', 'this', 'lil', 'lil', 'lil', 'rn', 'i', 'kinda', 'know', 'you', 'despise', '<EOS>']),
        array([-2.40578365, -2.42967963, -2.79503226, -3.8350358 , -3.99922132,
       -4.58851814, -3.67495298, -3.33830309, -4.33860493, -4.62443542,
       -4.99635792, -6.18057013, -6.7805233 , -7.7966733 , -7.9359169 ,
       -8.63088417, -8.73140526, -9.02580833], dtype=float32)]], dtype=object)

In [19]:
np.array([[data_formatting.decodeSent(q_0.predicted_ids.T[i][0], inv_map), 
          q_0.scores.T[i][0]]for i in range(inf_model_2_params['beam_width'])])

array([[['lmfao', 'i', 'can', 't', 'wait', 'to', 'the', 'hour', 'of',
         'tom', '<EOS>', '<EOS>'],
        ['-8.885274887084961', '-12.870537757873535', '-16.43918228149414',
         '-16.441207885742188', '-17.833393096923828',
         '-17.837202072143555', '-18.269447326660156',
         '-18.885576248168945', '-19.51644515991211', '-19.57634925842285',
         '-20.141868591308594', '-20.141868591308594']],

       [['lol', 'i', 'didn', 't', 'figure', 'about', 'die', 'at', 'this',
         'birthday', '?', '<EOS>'],
        ['-9.484236717224121', '-13.55785846710205', '-17.84384536743164',
         '-17.843873977661133', '-17.946748733520508',
         '-18.235815048217773', '-18.977397918701172',
         '-19.61638069152832', '-20.050893783569336', '-21.70056915283203',
         '-21.28431510925293', '-21.285179138183594']]],
      dtype='<U32')

In [ ]:
#data_formatting.decodeSent(q_0.beam_search_decoder_output.predicted_ids.T[0][0], inv_map)

In [9]:
def _shape(batch_size, from_shape):
    if not isinstance(from_shape, tf.TensorShape):
        return tensor_shape.TensorShape(None)
    else:
        batch_size = tf.contrib.util.constant_value(tf.convert_to_tensor(batch_size, name="batch_size"))
        return tf.TensorShape([batch_size]).concatenate(from_shape)

def _create_ta(s, d):
    return tf.TensorArray(
        dtype=d, size=0, dynamic_size=True, element_shape=_shape(batch_size, s)
        , clear_after_read=False)

In [10]:
time = tf.constant(0)

In [11]:
batch_size = inf_model_2.inference_decoder._batch_size
beam_width = inf_model_2.inference_decoder._beam_width
end_token = inf_model_2.inference_decoder._end_token
length_penalty_weight = inf_model_2.inference_decoder._length_penalty_weight
n_grams = pickle.load(open('n_grams_test.pkl', 'rb'))
vocab_size = inf_model_2_params['vocab_size']
beam_width = inf_model_2_params['beam_width']

name = None

In [12]:
#first_inputs
#These are the word embeddings of the initial token (which for initialize should be from the encoder?)
#Shape is [batch_size, embedding_size]
#initial_state
#This is the initial state of the decoder, a tuple consisting of 
#1. the LSTM cell and hiddens states
#2. The attention states

initial_finished, first_inputs, initial_state = inf_model_2.inference_decoder.initialize()

In [13]:
initial_outputs_ta = nest.map_structure(_create_ta, inf_model_2.inference_decoder.output_size,
                                                        inf_model_2.inference_decoder.output_dtype)

initial_sequence_lengths = tf.zeros_like(initial_finished, dtype=tf.int32)

In [14]:
#When we pass time !=0 to inference decoder, even with initial step results we get non-zero parent ids

#beam_search_output, beam_search_state, next_inputs, finished = \
#                                                inf_model_2.inference_decoder.step(time, first_inputs, initial_state)

In [15]:
#with ops.name_scope(name, "BeamSearchDecoderStep", (time, first_inputs, initial_state)):
    
cell_state = initial_state.cell_state

first_inputs = nest.map_structure(lambda inp: inf_model_2.inference_decoder._merge_batch_beams(inp, s=inp.shape[2:]), 
                                  first_inputs)

cell_state = nest.map_structure(inf_model_2.inference_decoder._maybe_merge_batch_beams, cell_state, 
                                inf_model_2.inference_decoder._cell.state_size)

In [16]:
##This goes through _cell first
#The output of cell_outputs is also present in the next_cell_state, the cell_outputs is the attention

#The cell_state includes, the cell and hidden states of the LSTM, the attention states, the alignment history, and timestep, 
#what does alignment history and attention states really mean? need to internalize...

cell_outputs, next_cell_state = inf_model_2.inference_decoder._cell(first_inputs, cell_state)

In [17]:
#These functions convert the shape of the cell outputs and states from [batch_size, beam_width, embedding_size]
#to [batch_size*beam_width, embedding_size], essentially, reducing a dimensionality by flattening the nested structure
#the map structure fn is used because cell state is a complex tuple structure, 
#outputs is a simpe structure (should be able to just call flatten?)
cell_outputs = nest.map_structure(lambda out: inf_model_2.inference_decoder._split_batch_beams(out, out.shape[1:]), 
                                  cell_outputs)

next_cell_state = nest.map_structure(inf_model_2.inference_decoder._maybe_split_batch_beams, next_cell_state, 
                                     inf_model_2.inference_decoder._cell.state_size)

#This is a dense layer that takes the cell output with dimension [beam_width, number of cells] to
#[beam_width, vocab_size]

if inf_model_2.inference_decoder._output_layer is not None:
    cell_outputs = inf_model_2.inference_decoder._output_layer(cell_outputs)

In [18]:
#cell_outputs_rnn_call, next_cell_state_rnn_call = inf_model_2.inference_decoder._cell.call(first_inputs, cell_state)

#cell_outputs_rnn_call_call, next_cell_state_rnn_call_call = inf_model_2.inference_decoder._cell.call(cell_outputs_rnn_call, 
#                                                                                                     next_cell_state_rnn_call)

In [19]:
static_batch_size = tensor_util.constant_value(batch_size)

prediction_lengths = initial_state.lengths
previously_finished = initial_state.finished

In [20]:
end_token = data_formatting.EOS

In [21]:
def _length_penalty(sequence_lengths, penalty_factor):
  """Calculates the length penalty. See https://arxiv.org/abs/1609.08144.
  Args:
    sequence_lengths: The sequence length of all hypotheses, a tensor
      of shape [beam_size, vocab_size].
    penalty_factor: A scalar that weights the length penalty.
  Returns:
    The length penalty factor, a tensor fo shape [beam_size].
  """
  penalty_factor = ops.convert_to_tensor(penalty_factor, name="penalty_factor")
  penalty_factor.set_shape(())  # penalty should be a scalar.
  static_penalty = tensor_util.constant_value(penalty_factor)
  if static_penalty is not None and static_penalty == 0:
    return 1.0
  return math_ops.div((5. + math_ops.to_float(sequence_lengths))
                      **penalty_factor, (5. + 1.)**penalty_factor)

def _check_maybe(t):
  if isinstance(t, tensor_array_ops.TensorArray):
    raise TypeError(
        "TensorArray state is not supported by BeamSearchDecoder: %s" % t.name)
  if t.shape.ndims is None:
    raise ValueError(
        "Expected tensor (%s) to have known rank, but ndims == None." % t)

def _maybe_tensor_gather_helper(gather_indices, gather_from, batch_size,
                                range_size, gather_shape):
  """Maybe applies _tensor_gather_helper.
  This applies _tensor_gather_helper when the gather_from dims is at least as
  big as the length of gather_shape. This is used in conjunction with nest so
  that we don't apply _tensor_gather_helper to inapplicable values like scalars.
  Args:
    gather_indices: The tensor indices that we use to gather.
    gather_from: The tensor that we are gathering from.
    batch_size: The batch size.
    range_size: The number of values in each range. Likely equal to beam_width.
    gather_shape: What we should reshape gather_from to in order to preserve the
      correct values. An example is when gather_from is the attention from an
      AttentionWrapperState with shape [batch_size, beam_width, attention_size].
      There, we want to preserve the attention_size elements, so gather_shape is
      [batch_size * beam_width, -1]. Then, upon reshape, we still have the
      attention_size as desired.
  Returns:
    output: Gathered tensor of shape tf.shape(gather_from)[:1+len(gather_shape)]
      or the original tensor if its dimensions are too small.
  """
  _check_maybe(gather_from)
  if gather_from.shape.ndims >= len(gather_shape):
    return _tensor_gather_helper(
        gather_indices=gather_indices,
        gather_from=gather_from,
        batch_size=batch_size,
        range_size=range_size,
        gather_shape=gather_shape)
  else:
    return gather_from

def _tensor_gather_helper(gather_indices, gather_from, batch_size,
                          range_size, gather_shape):
  """Helper for gathering the right indices from the tensor.
  This works by reshaping gather_from to gather_shape (e.g. [-1]) and then
  gathering from that according to the gather_indices, which are offset by
  the right amounts in order to preserve the batch order.
  Args:
    gather_indices: The tensor indices that we use to gather.
    gather_from: The tensor that we are gathering from.
    batch_size: The input batch size.
    range_size: The number of values in each range. Likely equal to beam_width.
    gather_shape: What we should reshape gather_from to in order to preserve the
      correct values. An example is when gather_from is the attention from an
      AttentionWrapperState with shape [batch_size, beam_width, attention_size].
      There, we want to preserve the attention_size elements, so gather_shape is
      [batch_size * beam_width, -1]. Then, upon reshape, we still have the
      attention_size as desired.
  Returns:
    output: Gathered tensor of shape tf.shape(gather_from)[:1+len(gather_shape)]
  """
  range_ = array_ops.expand_dims(math_ops.range(batch_size) * range_size, 1)
  gather_indices = array_ops.reshape(gather_indices + range_, [-1])
  output = array_ops.gather(
      array_ops.reshape(gather_from, gather_shape), gather_indices)
  final_shape = array_ops.shape(gather_from)[:1 + len(gather_shape)]
  static_batch_size = tensor_util.constant_value(batch_size)
  final_static_shape = (tensor_shape.TensorShape([static_batch_size])
                        .concatenate(
                            gather_from.shape[1:1 + len(gather_shape)]))
  output = array_ops.reshape(output, final_shape)
  output.set_shape(final_static_shape)
  return output
#Have to understand how this works!
def _mask_probs(probs, eos_token, finished):
  """Masks log probabilities.
  The result is that finished beams allocate all probability mass to eos and
  unfinished beams remain unchanged.
  Args:
    probs: Log probabiltiies of shape `[batch_size, beam_width, vocab_size]`
    eos_token: An int32 id corresponding to the EOS token to allocate
      probability to.
    finished: A boolean tensor of shape `[batch_size, beam_width]` that
      specifies which
      elements in the beam are finished already.
  Returns:
    A tensor of shape `[batch_size, beam_width, vocab_size]`, where unfinished
    beams stay unchanged and finished beams are replaced with a tensor with all
    probability on the EOS token.
  """
  vocab_size = array_ops.shape(probs)[2]
  finished_mask = array_ops.expand_dims(
      math_ops.to_float(1. - math_ops.to_float(finished)), 2)
  # These examples are not finished and we leave them
  non_finished_examples = finished_mask * probs
  # All finished examples are replaced with a vector that has all
  # probability on EOS
  finished_row = array_ops.one_hot(
      eos_token,
      vocab_size,
      dtype=probs.dtype,
      on_value=0.,
      off_value=probs.dtype.min)
  finished_examples = (1. - finished_mask) * finished_row
  return finished_examples + non_finished_examples

In [22]:
logits = cell_outputs
step_log_probs = nn_ops.log_softmax(logits)

In [23]:
step_log_probs = _mask_probs(step_log_probs, end_token, previously_finished)

In [24]:
total_probs = array_ops.expand_dims(initial_state.log_probs, 2) + step_log_probs

In [25]:
vocab_size = logits.shape[-1].value or array_ops.shape(logits)[-1]

lengths_to_add = array_ops.one_hot(
      indices=array_ops.tile(
          array_ops.reshape(end_token, [1, 1]), [batch_size, beam_width]),
      depth=vocab_size,
      on_value=0,
      off_value=1)

In [26]:
def anti_lm_body(test, n_grams_tf, beam_pad, current_beam, n_beams):
    #This loops on each beam individually
      
    def grab_probs(n_grams_tf, y_equal_2): 
    
        #print ('Grab Probabilities')
        y_args = tf.where(y_equal_2)

        #Grab the n_gram sequences that are matched
        y_diff_gather = tf.gather(n_grams_tf, y_args)
        #Take the last token in each sequence and count their unique occurrences

        last_token_pos = tf.shape(y_diff_gather)[-1]

        y_last_token = y_diff_gather[:,0][:,last_token_pos-1]
        y_last_token_unique = tf.unique_with_counts(y_last_token)

        total_count = tf.reduce_sum(y_last_token_unique.count)

        indices = tf.reshape(y_last_token_unique.y, [tf.shape(y_last_token_unique.y)[0], 1])

        test_add_result = tf.scatter_nd(indices=indices, 
                    updates=y_last_token_unique.count, shape=scatter_base)/total_count

        test_add_result = tf.log(test_add_result + 10e-10)

        return tf.reshape(test_add_result, [1, vocab_size])

    def dump_zeros(n_grams_tf, y_equal_2): 
        #print ('No matches found')
        return tf.constant([[0. for i in range(vocab_size)]], dtype=tf.float64)
    
    scatter_base = tf.constant([vocab_size]) #Size of dict for scatter base will specify at run time

    #Find where current beam matches n_gram sequence up to current seq pos, cast as int
    y_test = tf.to_int32(tf.equal(n_grams_tf, beam_pad[current_beam]))
    #Reduce across the length of the beam 
    y_test_reduce_sum = tf.reduce_sum(y_test, axis=1)

    y_empty = tf.reduce_sum(y_test_reduce_sum)

    #Find args where the beam is matched to the n_gram combinations
    y_equal_2  = tf.equal(time, y_test_reduce_sum)

    #Why does cond proceed down both paths? okay, understand this now...
    test_add = tf.cond(tf.equal(0, tf.reduce_sum(tf.to_int32(y_equal_2))),
        true_fn=lambda: dump_zeros(n_grams_tf, y_equal_2),
        false_fn=lambda : grab_probs(n_grams_tf, y_equal_2))

    test = tf.concat([test, test_add], axis=0)
    
    return test, n_grams_tf, beam_pad, current_beam+1, n_beams

def anti_lm_condition(test, n_grams_tf, beam_pad, current_beam, n_beams):
    return current_beam < n_beams #number of beams, i.e. beam width, will specify at run time

In [27]:
def _get_scores(log_probs, sequence_lengths, length_penalty_weight, time):
    """Calculates scores for beam search hypotheses.
    Args:
    log_probs: The log probabilities with shape
    `[batch_size, beam_width, vocab_size]`.
    sequence_lengths: The array of sequence lengths.
    length_penalty_weight: Float weight to penalize length. Disabled with 0.0.
    Returns:
    The scores normalized by the length_penalty.
    """
    
    def fn1(): return tf.constant(n_grams[1])
    def fn2(): return tf.constant(n_grams[2])
    def fn3(): return tf.constant(n_grams[3])
    def fn_default(): return tf.constant(-1)
        
    def time_zero_anti_lm(score):
        n_grams_tf = tf.constant(n_grams[0])
        scatter_base = tf.constant([vocab_size]) #Size of dict for scatter base will specify at run time

        y_unique_with_counts = tf.unique_with_counts(tf.reshape(n_grams_tf, [1, tf.shape(n_grams_tf)[0]])[0])

        total_count = tf.reduce_sum(y_unique_with_counts.count)
        
        indices = tf.reshape(y_unique_with_counts.y, [tf.shape(y_unique_with_counts.y)[0], 1])

        test_add = tf.scatter_nd(indices=indices, 
                    updates=y_unique_with_counts.count, shape=scatter_base)/total_count

        #Add small value for numerically stability
        test_add = tf.log(test_add + 10e-10)    

        test_add_tile = tf.reshape(tf.tile(test_add, multiples=[beam_width]), shape=[1, beam_width, vocab_size])
        
        score = score + tf.to_float(test_add_tile)
        
        return score, n_grams_tf
    
    def time_not_zero_anti_lm(score):
        #No anti-lm correction past 4th sequence position
        n_grams_tf = tf.case({tf.equal(time,1): fn1, 
                     tf.equal(time,2): fn2,
                     tf.equal(time,3): fn3}, default=fn_default, exclusive=True)

        #When you specify the axis of concat such a rank must exist! 
        #Thus cannot specify axis=1 if the rank is 0!

        concat_base = tf.constant([[1.0 for i in range(vocab_size)]], dtype=tf.float64) #concat base will specify vocab size at run time

        beam = tf.transpose(initial_outputs_ta.predicted_ids.concat())

        beam_pad = tf.pad(beam, [[0, 0], [0, 1]], mode='CONSTANT', constant_values=0)

        initial_beam_step = tf.constant(0)  #This starting the loop from the first beam

        n_beams = tf.constant(beam_width) #will specify at run time
        
        anti_lm_outputs = tf.while_loop(anti_lm_condition, anti_lm_body, 
                        [concat_base, n_grams_tf, beam_pad, initial_beam_step, n_beams], 
                        shape_invariants=[tf.TensorShape([None, vocab_size]), 
                                          n_grams_tf.get_shape(),
                                      beam_pad.get_shape(), initial_beam_step.get_shape(), n_beams.get_shape()])

        anti_beam_probs = anti_lm_outputs[0][1:]
        score = score + tf.to_float(anti_beam_probs)
        
        return score, n_grams_tf
    
    length_penality_ = tf_helpers._length_penalty(sequence_lengths=sequence_lengths, penalty_factor=length_penalty_weight)

    score = log_probs/length_penality_
    
    score, n_grams_tf = tf.cond(tf.equal(0, time), 
                                lambda : time_zero_anti_lm(score), lambda: time_not_zero_anti_lm(score))
    
    return score

In [28]:
import tf_helpers

In [29]:
add_mask = (1 - math_ops.to_int32(previously_finished))

lengths_to_add = array_ops.expand_dims(add_mask, 2) * lengths_to_add

new_prediction_lengths = (lengths_to_add + array_ops.expand_dims(prediction_lengths, 2))

In [30]:
def score_total_probs(total_probs):
    return total_probs

scores = tf.cond(time < 4, 
                 true_fn=lambda: _get_scores(log_probs=total_probs,
                 sequence_lengths=new_prediction_lengths, length_penalty_weight=0.5, time=time), 
                 false_fn=lambda: score_total_probs(total_probs) )


In [31]:
input_user = data_formatting.encodeSent(['i', 'll', 'be', 'there', 'tomorrow', '<EOS>'], vocab_dict)
with tf.Session() as sess:
    #sess.run(tf.global_variables_initializer())
    #tf.set_random_seed(1)
    saver = tf.train.Saver()
   
    saver.restore(sess, '../chkpt/seq2seq_twitter_testing-5501')
    q_0 = sess.run([scores], 
                 feed_dict={'training_model/encoder_inputs:0':[input_user], 
                         'training_model/encoder_inputs_length:0':[len(input_user)]})

INFO:tensorflow:Restoring parameters from ../chkpt/seq2seq_twitter_testing-5501


In [33]:
#scores = total_probs + p_T
scores_x = _get_scores(log_probs=total_probs,
                         sequence_lengths=new_prediction_lengths,
                          length_penalty_weight=0, time=time)
scores_y = total_probs

In [34]:
scores = scores_x

In [35]:
scores_shape = array_ops.shape(scores)

time = ops.convert_to_tensor(time, name="time")

scores_flat = control_flow_ops.cond(time > 0,
      lambda: array_ops.reshape(scores, [batch_size, -1]), lambda: scores[:, 0])

num_available_beam = control_flow_ops.cond(
      time > 0, lambda: math_ops.reduce_prod(scores_shape[1:]),
      lambda: math_ops.reduce_prod(scores_shape[2:]))

In [36]:
# Pick the next beams according to the specified successors function
next_beam_size = math_ops.minimum(
  ops.convert_to_tensor(beam_width, dtype=dtypes.int32, name="beam_width"),
  num_available_beam)

next_beam_scores, word_indices = nn_ops.top_k(scores_flat, k=next_beam_size)
next_beam_scores.set_shape([static_batch_size, beam_width])
word_indices.set_shape([static_batch_size, beam_width])

In [37]:
next_beam_probs = _tensor_gather_helper(
      gather_indices=word_indices,
      gather_from=scores,
      batch_size=batch_size,
      range_size=beam_width * vocab_size,
      gather_shape=[-1])

#Here, word indices represents the positions of the flattened list of beams, which goes from 1 to beam_width*vocab_size
#thus the word indices obtained are not the word indices of the 1 to N vocab but the flattened out list of logprobs from 1 to 
#beam_width*vocab_size
next_word_ids = math_ops.to_int32(word_indices % vocab_size)
#The beam_ids represent which beams these word indices (and thereby log prob values belong to), as
#we have word indices extracted out of 1 to beam_width*vocab_size
next_beam_ids = math_ops.to_int32(word_indices / vocab_size)

In [38]:
input_user = data_formatting.encodeSent(['i', 'll', 'be', 'there', 'tomorrow', '<EOS>'], vocab_dict)
with tf.Session() as sess:
    #sess.run(tf.global_variables_initializer())
    #tf.set_random_seed(1)
    saver = tf.train.Saver()
   
    saver.restore(sess, '../chkpt/seq2seq_twitter_testing-5501')
    q_0 = sess.run([next_word_ids, next_beam_ids, scores, scores_flat, next_beam_scores, next_beam_probs,
                    word_indices, next_beam_size], 
                 feed_dict={'training_model/encoder_inputs:0':[input_user], 
                         'training_model/encoder_inputs_length:0':[len(input_user)]})

INFO:tensorflow:Restoring parameters from ../chkpt/seq2seq_twitter_testing-5501


InvalidArgumentError: Invalid reduction dimension (1 for input with 1 dimension(s)
	 [[Node: cond_1/while/Sum = Sum[T=DT_INT32, Tidx=DT_INT32, keep_dims=false, _device="/job:localhost/replica:0/task:0/gpu:0"](cond_1/while/ToInt32/_185, cond_1/while/Sum/reduction_indices)]]
	 [[Node: training_model/BahdanauAttention/strided_slice_3/_305 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_827_training_model/BahdanauAttention/strided_slice_3", tensor_type=DT_INT32, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'cond_1/while/Sum', defined at:
  File "c:\users\euix\anaconda3\envs\tensorflow-gpu\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\users\euix\anaconda3\envs\tensorflow-gpu\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "c:\users\euix\anaconda3\envs\tensorflow-gpu\lib\sit

In [53]:
previously_finished = _tensor_gather_helper(
      gather_indices=next_beam_ids,
      gather_from=previously_finished,
      batch_size=batch_size,
      range_size=beam_width,
      gather_shape=[-1])

next_finished = math_ops.logical_or(previously_finished, math_ops.equal(next_word_ids, end_token))

In [54]:
lengths_to_add = math_ops.to_int32(math_ops.not_equal(next_word_ids, end_token))
lengths_to_add = (1 - math_ops.to_int32(next_finished)) * lengths_to_add

In [55]:
next_prediction_len = _tensor_gather_helper(
      gather_indices=next_beam_ids,
      gather_from=initial_state.lengths,
      batch_size=batch_size,
      range_size=beam_width,
      gather_shape=[-1])

next_prediction_len += lengths_to_add

In [131]:
next_cell_state = nest.map_structure(
      lambda gather_from: _maybe_tensor_gather_helper(
          gather_indices=next_beam_ids,
          gather_from=gather_from,
          batch_size=batch_size,
          range_size=beam_width,
          gather_shape=[batch_size * beam_width, -1]),
      next_cell_state)

In [132]:
with tf.Session() as sess:
    saver = tf.train.Saver()
   
    saver.restore(sess, '../chkpt/seq2seq_twitter_testing-5501')
    q_1 = sess.run([next_cell_state],
                     feed_dict={'training_model/encoder_inputs:0':[input_user], 
                         'training_model/encoder_inputs_length:0':[len(input_user)]})    

INFO:tensorflow:Restoring parameters from ../chkpt/seq2seq_twitter_testing-5501


In [133]:
q_1

[(LSTMStateTuple(c=array([[[-0.36518568,  0.15221889,  1.43084002,  1.26691902,  0.97570741,
         -1.65228498,  0.2762605 ,  0.70097274, -0.82302231, -0.39878359,
         -0.47339875,  0.56113648,  1.45701504, -0.58288693,  0.2458801 ,
          0.45721632,  1.06034863, -0.53341776,  0.84067792, -2.14312649,
         -0.3462446 , -2.72415495, -0.09161211,  1.36289251, -0.41485611,
         -0.77971447, -1.01373613, -0.1626246 , -0.42198175,  0.79769164,
         -1.44223773, -0.12655784, -1.03397453, -0.45727509,  0.49624717,
         -0.71556503, -0.1682298 , -0.59325242, -0.70525527, -1.19461775,
         -0.71584553,  0.53623444, -0.3736887 ,  1.55030262,  1.60065281,
          0.76959068, -0.15846567, -1.26996934, -0.51451391,  3.21015644,
          0.56469333, -0.01150133, -0.63608539,  1.63232625,  0.79776645,
         -1.23871732,  1.50188172, -2.6774683 , -0.78931975,  2.26646423,
         -0.78553116, -0.80997407, -2.48717618,  0.68119168,  0.8132056 ,
         -1.2882286

In [134]:
#These are the returns, next_state and output to the step function, but how exactly do we transpose this back and push it
#into cell? Moreover what is teh mechanics that determines when the beams become different? I.E. are no longer identical

In [135]:
beam_search_state = tf.contrib.seq2seq.BeamSearchDecoderState(
      cell_state=next_cell_state,
      log_probs=next_beam_probs,
      lengths=next_prediction_len,
      finished=next_finished)

beam_search_output = tf.contrib.seq2seq.BeamSearchDecoderOutput(
      scores=next_beam_scores,
      predicted_ids=next_word_ids,
      parent_ids=next_beam_ids)

In [136]:
#Here we define the returns
finished = beam_search_state.finished
next_finished = finished
sample_ids = beam_search_output.predicted_ids

In [137]:
#This part is most important, look up the new seq terms and take their embedding
#We pass that onto the decoder cell network in the next loop iteration

next_inputs = tf.cond(tf.reduce_all(finished), lambda: inf_model_2.inference_decoder._start_inputs,
              lambda: inf_model_2.inference_decoder._embedding_fn(sample_ids))

In [138]:
#Here we take the initially created tensor array with no output values and write the beam search output onto that tensor array 

initial_outputs_ta = nest.map_structure(lambda ta, out: ta.write(time, out),
                          initial_outputs_ta, beam_search_output)

In [139]:
final_outputs = nest.map_structure(lambda ta: ta.stack(), initial_outputs_ta)

In [140]:
next_sequence_lengths = tf.where(tf.logical_and(tf.logical_not(finished), next_finished),
      tf.fill(tf.shape(initial_sequence_lengths), time + 1), initial_sequence_lengths)

In [141]:
finished = beam_search_state.finished

In [142]:
sample_ids = beam_search_output.predicted_ids

In [143]:
next_inputs = control_flow_ops.cond(math_ops.reduce_all(finished), 
                                    lambda: inf_model_2.inference_decoder._start_inputs,
                                    lambda: inf_model_2.inference_decoder._embedding_fn(sample_ids))

In [144]:
#next_inputs
#These are the chosen word ids which we have performed the embedding look up for that we pass on
first_inputs = next_inputs

#next_states
#these are the next states of the decoder network which pass back through the process
initial_state = beam_search_state

initial_finished = finished

In [145]:
#initial_outputs_ta = nest.map_structure(_create_ta, inf_model_2.inference_decoder.output_size,
#                                                        inf_model_2.inference_decoder.output_dtype)

initial_sequence_lengths = next_sequence_lengths

In [146]:
time = tf.add(time, 1)

In [121]:
vocab_size = 17322

import pandas as pd

import tensorflow as tf
import numpy as np

import pickle
tf.reset_default_graph()
time = 1
n_grams = pickle.load(open('n_grams_test.pkl', 'rb'))

n_grams_tf = tf.constant(n_grams[time])

#beam = tf.constant([[6178, 5729], [8847, 16625], [12377,  6605]])
beam = tf.constant([[ 6171231],
        [ 4400],
        [10359],
        [11678]])
beam_pad = tf.pad(beam, [[0, 0], [0, 1]], mode='CONSTANT', constant_values=0)

initial_beam_step = tf.constant(0)
n_beams = tf.constant(4)

In [218]:
df = pd.DataFrame(n_grams[time], columns=['a', 'b'])

In [219]:
#When you specify the axis of concat such a rank must exist! 
#Thus cannot specify axis=1 if the rank is 0!
scatter_base = tf.constant([vocab_size])
concat_base = tf.constant([[1.0 for i in range(vocab_size)]], dtype=tf.float64)

In [220]:
def grab_probs(n_grams_tf, y_equal_2): 
    #print ('Grab Probabilities')
    y_args = tf.where(y_equal_2)

    #Grab the n_gram sequences that are matched
    y_diff_gather = tf.gather(n_grams_tf, y_args)
    #Take the last token in each sequence and count their unique occurrences

    last_token_pos = tf.shape(y_diff_gather)[-1]

    y_last_token = y_diff_gather[:,0][:,last_token_pos-1]
    y_last_token_unique = tf.unique_with_counts(y_last_token)
        
    total_count = tf.reduce_sum(y_last_token_unique.count)
    
    indices = tf.reshape(y_last_token_unique.y, [tf.shape(y_last_token_unique.y)[0], 1])
    
    test_add_result = tf.scatter_nd(indices=indices, 
                updates=y_last_token_unique.count, shape=scatter_base)/total_count

    test_add_result = tf.log(test_add_result + 10e-10)
    
    return tf.reshape(test_add_result, [1, 17322])

def dump_zeros(n_grams_tf, y_equal_2): 
    #print ('No matches found')
    return tf.constant([[0. for i in range(vocab_size)]], dtype=tf.float64)

#Find where current beam matches n_gram sequence up to current seq pos, cast as int
y_test = tf.to_int32(tf.equal(n_grams_tf, beam_pad[initial_beam_step]))
#Reduce across the length of the beam 
y_test_reduce_sum = tf.reduce_sum(y_test, axis=1)

y_empty = tf.reduce_sum(y_test_reduce_sum)

#Find args where the beam is matched to the n_gram combinations
y_equal_2  = tf.equal(time, y_test_reduce_sum)

#Why does cond proceed down both paths?
test_add = tf.cond(tf.equal(0, tf.reduce_sum(tf.to_int32(y_equal_2))),
    true_fn=lambda: dump_zeros(n_grams_tf, y_equal_2),
    false_fn=lambda : grab_probs(n_grams_tf, y_equal_2))

In [223]:
with tf.Session() as sess:
    q = sess.run([tf.equal(0, tf.reduce_sum(tf.to_int32(y_equal_2))), test_add])

In [224]:
q

[True, array([[ 0.,  0.,  0., ...,  0.,  0.,  0.]])]

In [225]:
np.unique(q[1][0])

array([ 0.])

In [226]:
concat_base = tf.concat([concat_base, test_add], axis=0)

In [227]:
with tf.Session() as sess:
    q = sess.run([concat_base])

In [228]:
q[0]

array([[ 1.,  1.,  1., ...,  1.,  1.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [1]:
def anti_lm_process_body(test, n_grams_tf, beam_pad, current_beam, n_beams):
    #This loops on each beam individually
    
    #test = tf.concat([test, beam[time]], axis=0)
    
    #Find where current beam matches n_gram sequence up to current seq pos, cast as int
    y_test = tf.to_int32(tf.equal(n_grams_tf, beam_pad[current_beam]))
    #Reduce across the length of the beam 
    y_test_reduce_sum = tf.reduce_sum(y_test, axis=1)
    #Find args where the beam is matched to the n_gram combinations
    y_args = tf.where(tf.equal(2, y_test_reduce_sum))
    #Grab the n_gram sequences that are matched
    y_diff_gather = tf.gather(n_grams_tf, y_args)
    #Take the last token in each sequence and count their unique occurrences
    y_last_token = y_diff_gather[:,0][:,time]
    y_last_token_unique = tf.unique_with_counts(y_last_token)
    
    total_count = tf.reduce_sum(y_last_token_unique.count)
    test_add = tf.scatter_nd(indices=y_last_token_unique.y, 
                    updates=y_last_token_unique.count, shape=scatter_base)/total_count
    
    test_add = tf.log(test_add + 10e-10)
    
    test = tf.concat([test, [test_add]], axis=0)

    return test, n_grams_tf, beam_pad, current_beam+1, n_beams

def anti_lm_process_body_condition(test, n_grams_tf, beam_pad, current_beam, n_beams):
    return current_beam < 3

outputs = tf.while_loop(anti_lm_process_body_condition, anti_lm_process_body, 
                        [concat_base, n_grams_tf, beam_pad, initial_beam_step, n_beams], 
                        shape_invariants=[tf.TensorShape([None, 17322]), n_grams_tf.get_shape(),
                                      beam_pad.get_shape(), initial_beam_step.get_shape(), n_beams.get_shape()])

anti_beam_probs = outputs[0][1:]

with tf.Session() as sess:
    tf.initialize_all_variables().run()
    #q = sess.run(outputs[0].stack())
    q = sess.run(anti_beam_probs)

In [309]:
time

<tf.Tensor 'Add:0' shape=() dtype=int32>

In [307]:
with tf.Session() as sess:
    print(sess.run(time))

1


In [302]:
time = tf.constant(0)